# 모델 매개변수 최적화
이제 모델과 데이터가 준비됐으니, 데이터에 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트 할 차례

모델을 학습하는 과정은 반복적인 과정을 거친다.

각 에폭에서 모델은 출력을 추측하고, 추측과 정답 사이의 loss를 계산하고, 매개변수에 대한 오류의 derivative를 수집한 뒤, 경사하강법을 사용하여 이 파라미터들을 optimize(최적화)한다.

이 과정에 대한 자세한 설명은 아래의 링크를 참고

https://www.youtube.com/watch?v=tIeHLnjs5U8




# 기본 코드

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

C:\Users\JIYEON\anaconda3\envs\torch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Hyperparameter
하이퍼 파라미터는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수
서로 다른 하이퍼 파라미터 값은 모델 학습과 convergence rate(수렴율)에 영향을 미칠 수 있다.

학습 시에는 다음과 같은 하이퍼 파라미터를 정의한다.
1. epoch : 데이터셋 반복하는 횟수
2. batch size : 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플 수
3. learning rate : 각 배치/에폭에서 모델의 매개변수를 조절하는 비율.
-> 값이 작을수록 학습 속도가 느리고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있다.

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# 최적화 단계(Optimization Loop)
하이퍼 파라미터를 설정한 뒤에는 optimizer를 통해 모델을 학습하고 최적화 할 수 있다.

최적화 단계 각 iteration을 epoch라고 부른다.


하나의 epoch는 다음 두 부분으로 구성된다.
1. train loop : 학습용 데이터셋을 iterate하고 최적의 매개변수로 수렴한다.
2. val/test loop : 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복

training loop에서 일어나는 몇 가지 개념들을 간략히 살펴보자.

# loss function
학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높다.

loss function은 획득한 결과와 실제 값 사이의 차이를 측정하며, 학습 중에 이 값을 최소화하려고 한다.

주어진 데이터 샘플을 입력으로 계산한 예측과 label을 비교하여 loss를 계산



일반적인 손실함수에는 regression task에 사용하는 nn.MSELoss

classification에 사용하는 nn.NLLLoss(Negative Log Likelihood)

nn.LogSoftmax와 nn.NLLLoss를 합친 nn.CrossEntropyLoss 등이 있다.

